In [117]:
import numpy as np
import yaml
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl

from captum.attr import IntegratedGradients, GuidedGradCam
from captum.attr import LayerGradCam

from contrastive.models.contrastive_learner_fusion import ContrastiveLearnerFusion
from contrastive.data.datamodule import DataModule_Learning
import omegaconf

import torchvision.transforms as transforms
from contrastive.augmentations import *

In [2]:
def transform_nothing_done():
    return \
        transforms.Compose([
            SimplifyTensor(),
            BinarizeTensor(),
            EndTensor()
        ])

## load trained classifier

In [3]:
sub_dir = "/neurospin/dico/jlaval/Runs/01_deep_supervised/Program/Output/2023-07-11/11-39-50/"

config = omegaconf.OmegaConf.load(sub_dir+'/.hydra/config.yaml')

# input size must be transformed from str to tuple
for reg in range(len(config.data)):
        config.data[reg].input_size = eval(config.data[reg].input_size)

In [4]:
data_module = DataModule_Learning(config)
    
model = ContrastiveLearnerFusion(config,
                                 sample_data=data_module)

INFO:torch.distributed.nn.jit.instantiator: Created a temporary directory at /tmp/tmplsnk1t4z
INFO:torch.distributed.nn.jit.instantiator: Writing /tmp/tmplsnk1t4z/_remote_module_non_scriptable.py
INFO:contrastive_learner_fusion.py: n_datasets 1
INFO:contrastive_learner_fusion.py: activation = linear


In [5]:
model.load_pretrained_model(config.pretrained_model_path,
                            encoder_only=config.load_encoder_only)

INFO:contrastive_learner_fusion.py: Layers not loaded = ['converter.0.weight', 'converter.0.bias', 'projection_head.layers.Linear0.weight', 'projection_head.layers.Linear0.bias', 'projection_head.layers.Linear1.weight', 'projection_head.layers.Linear1.bias']


In [6]:
model.eval()

ContrastiveLearnerFusion(
  (backbones): ModuleList(
    (0): ConvNet(
      (encoder): Sequential(
        (conv0): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (norm0): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (LeakyReLU0): LeakyReLU(negative_slope=0.01)
        (DropOut0): Dropout3d(p=0.05, inplace=False)
        (conv0a): Conv3d(16, 16, kernel_size=(4, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
        (norm0a): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (LeakyReLU0a): LeakyReLU(negative_slope=0.01)
        (DropOut0a): Dropout3d(p=0.05, inplace=False)
        (conv1): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (norm1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (LeakyReLU1): LeakyReLU(negative_slope=0.01)
        (DropOut1): Dropout3d(p=0.05, inplace=False)
        (con

In [7]:
# define a new model with the layers of both model.ConvNet and model.projection_head ?
# is it required to unfold the model ?

## Prepare skeletons for model input

In [106]:
skels = np.load(config.dataset[list(config.dataset.keys())[0]]['numpy_all'])
skels = skels.astype(float)
print(np.sum(skels))

26152045.0


In [107]:
transform = transform_nothing_done()
skels = transform(torch.tensor(skels))
skels.requires_grad=True
skels = torch.swapaxes(skels, 0, 1)
skels = skels.float()
print(torch.sum(skels))
print(skels.shape)

tensor(504152., grad_fn=<SumBackward0>)
torch.Size([374, 1, 22, 49, 46])


## Forward in model

In [115]:
model(skels.unsqueeze(0))

tensor([[ 7.0136e-03, -1.7261e-01],
        [-2.7688e-02, -2.6911e-01],
        [-1.5754e-02, -2.2051e-01],
        [ 1.5490e-02, -4.4052e-01],
        [-4.4384e-02, -1.2714e-01],
        [-8.1747e-03, -3.2129e-01],
        [ 1.8528e-02, -2.5711e-01],
        [ 6.7334e-03, -2.7972e-01],
        [-2.2970e-02, -3.2176e-01],
        [-3.2935e-02, -8.3741e-02],
        [-3.7247e-03, -4.3459e-01],
        [ 4.0524e-02, -3.1692e-01],
        [-3.0525e-02, -1.6876e-01],
        [ 2.4083e-02, -3.7960e-01],
        [ 2.8154e-02, -1.6546e-01],
        [ 2.6277e-02, -3.2448e-01],
        [-6.8338e-03, -2.5122e-01],
        [ 4.0768e-02, -1.8313e-01],
        [-1.7928e-02, -4.6127e-02],
        [ 5.3741e-02, -1.0710e-01],
        [ 8.9630e-03, -4.2657e-01],
        [-4.2315e-02, -1.1255e-01],
        [-7.3367e-03, -2.8975e-01],
        [-7.1510e-03, -3.1072e-01],
        [ 2.6100e-02, -4.5674e-01],
        [ 5.9169e-02, -2.4069e-01],
        [ 3.9917e-03, -2.3600e-01],
        [-1.8286e-02, -3.473

In [116]:
model.forward(skels.unsqueeze(0))

tensor([[ 7.0136e-03, -1.7261e-01],
        [-2.7688e-02, -2.6911e-01],
        [-1.5754e-02, -2.2051e-01],
        [ 1.5490e-02, -4.4052e-01],
        [-4.4384e-02, -1.2714e-01],
        [-8.1747e-03, -3.2129e-01],
        [ 1.8528e-02, -2.5711e-01],
        [ 6.7334e-03, -2.7972e-01],
        [-2.2970e-02, -3.2176e-01],
        [-3.2935e-02, -8.3741e-02],
        [-3.7247e-03, -4.3459e-01],
        [ 4.0524e-02, -3.1692e-01],
        [-3.0525e-02, -1.6876e-01],
        [ 2.4083e-02, -3.7960e-01],
        [ 2.8154e-02, -1.6546e-01],
        [ 2.6277e-02, -3.2448e-01],
        [-6.8338e-03, -2.5122e-01],
        [ 4.0768e-02, -1.8313e-01],
        [-1.7928e-02, -4.6127e-02],
        [ 5.3741e-02, -1.0710e-01],
        [ 8.9630e-03, -4.2657e-01],
        [-4.2315e-02, -1.1255e-01],
        [-7.3367e-03, -2.8975e-01],
        [-7.1510e-03, -3.1072e-01],
        [ 2.6100e-02, -4.5674e-01],
        [ 5.9169e-02, -2.4069e-01],
        [ 3.9917e-03, -2.3600e-01],
        [-1.8286e-02, -3.473

In [108]:
skels_copy = skels.clone()
skels_copy = skels_copy.detach().numpy()
baseline = torch.tensor(np.array([skels_copy[0] for k in range(374)]))
baseline.requires_grad=True
baseline = baseline.float()
print(torch.sum(baseline))
print(baseline.shape)

tensor(447304., grad_fn=<SumBackward0>)
torch.Size([374, 1, 22, 49, 46])


In [109]:
ig = IntegratedGradients(model)
attributions, delta = ig.attribute(skels.unsqueeze(0), baseline.unsqueeze(0), target=0, return_convergence_delta=True)
print('IG Attributions:', attributions)
print('Convergence Delta:', delta)

IG Attributions: tensor([[[[[[-0., -0., 0.,  ..., -0., 0., -0.],
            [-0., -0., -0.,  ..., -0., -0., -0.],
            [0., -0., -0.,  ..., -0., -0., -0.],
            ...,
            [-0., -0., 0.,  ..., -0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.]],

           [[-0., -0., -0.,  ..., 0., 0., -0.],
            [0., -0., -0.,  ..., -0., 0., -0.],
            [0., -0., -0.,  ..., 0., 0., 0.],
            ...,
            [-0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [0., 0., 0.,  ..., 0., 0., 0.]],

           [[0., -0., -0.,  ..., -0., -0., -0.],
            [0., 0., 0.,  ..., -0., 0., -0.],
            [0., -0., -0.,  ..., -0., -0., -0.],
            ...,
            [-0., 0., 0.,  ..., -0., 0., -0.],
            [0., 0., 0.,  ..., 0., 0., 0.],
            [-0., 0., 0.,  ..., 0., 0., -0.]],

           ...,

           [[0., 0., 0.,  ..., 0., 0., -0.],
            [0., 0., -0., 

In [112]:
print(torch.sum(torch.abs(attributions)))
print(torch.max(torch.abs(attributions)))
# probably not well fitted ...

tensor(0.3026, dtype=torch.float64, grad_fn=<SumBackward0>)
tensor(6.7648e-06, dtype=torch.float64, grad_fn=<MaxBackward1>)


In [113]:
GC = GuidedGradCam(model, model.backbones[0].encoder.conv2a)
attributions = GC.attribute(skels.unsqueeze(0), target=0) # no need to give target for single output. But 2 outputs in our case ?
print('GC Attributions:', attributions)

/volatile/jl274628/Runs/venv_local/lib/python3.7/site-packages/captum/attr/_core/guided_backprop_deconvnet.py:65: UserWarning: Setting backward hooks on ReLU activations.The hooks will be removed after the attribution is finished
  "Setting backward hooks on ReLU activations."
/volatile/jl274628/Runs/venv_local/lib/python3.7/site-packages/captum/attr/_core/guided_grad_cam.py:217: UserWarning: Couldn't appropriately interpolate GradCAM attributions for some input tensors, returning empty tensor for corresponding attributions.
  "Couldn't appropriately interpolate GradCAM attributions for some "


GC Attributions: tensor([])


In [118]:
GC = LayerGradCam(model, model.backbones[0].encoder.conv2a)
attributions = GC.attribute(skels.unsqueeze(0), target=0) # no need to give target for single output. But 2 outputs in our case ?
print('GC Attributions:', attributions)

GC Attributions: tensor([[[[[-2.7444e-04, -7.1144e-05,  1.5250e-04, -6.8057e-04, -6.3341e-04],
           [-5.3678e-05,  1.3242e-04, -9.3661e-04, -7.3804e-04, -3.7601e-04],
           [ 2.4485e-04, -5.4113e-04, -6.8364e-04,  2.8609e-04, -4.6864e-04],
           [-4.3344e-04,  2.8516e-04,  1.0071e-03,  2.2282e-03,  3.2907e-04],
           [-7.3195e-04, -4.8285e-05,  9.3910e-04, -4.3187e-04, -2.3227e-04],
           [-1.7226e-04, -2.9939e-04,  6.7572e-05, -6.0677e-04, -6.2035e-04]],

          [[ 4.0129e-05, -7.5015e-05, -6.1950e-05, -7.0265e-04, -7.1800e-04],
           [-3.4586e-05,  1.7949e-04,  1.2467e-03,  3.6672e-04,  7.1490e-04],
           [-8.9473e-05, -1.1735e-04,  1.4638e-03,  3.7462e-04, -3.6484e-04],
           [ 6.5423e-04, -1.6053e-04,  1.8400e-03,  1.4375e-03,  9.0571e-05],
           [ 2.2230e-04, -4.0170e-04,  5.9775e-05, -7.3462e-04, -3.5417e-04],
           [ 7.4459e-04, -8.4129e-04, -1.1798e-04, -7.6873e-04, -5.2984e-04]]]],



        [[[[ 2.8772e-04,  1.5594e-04,  

In [122]:
attributions[0].shape

torch.Size([1, 2, 6, 5])

In [124]:
GC = LayerGradCam(model, model.backbones[0].encoder.conv2a)
attributions = GC.attribute(skels.unsqueeze(0), target=0, relu_attributions=True) # no need to give target for single output. But 2 outputs in our case ?
print('GC Attributions:', attributions)

GC Attributions: tensor([[[[[0.0000e+00, 0.0000e+00, 1.5250e-04, 0.0000e+00, 0.0000e+00],
           [0.0000e+00, 1.3242e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00],
           [2.4485e-04, 0.0000e+00, 0.0000e+00, 2.8609e-04, 0.0000e+00],
           [0.0000e+00, 2.8516e-04, 1.0071e-03, 2.2282e-03, 3.2907e-04],
           [0.0000e+00, 0.0000e+00, 9.3910e-04, 0.0000e+00, 0.0000e+00],
           [0.0000e+00, 0.0000e+00, 6.7572e-05, 0.0000e+00, 0.0000e+00]],

          [[4.0129e-05, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
           [0.0000e+00, 1.7949e-04, 1.2467e-03, 3.6672e-04, 7.1490e-04],
           [0.0000e+00, 0.0000e+00, 1.4638e-03, 3.7462e-04, 0.0000e+00],
           [6.5423e-04, 0.0000e+00, 1.8400e-03, 1.4375e-03, 9.0571e-05],
           [2.2230e-04, 0.0000e+00, 5.9775e-05, 0.0000e+00, 0.0000e+00],
           [7.4459e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]]]],



        [[[[2.8772e-04, 1.5594e-04, 2.7853e-04, 0.0000e+00, 0.0000e+00],
           [4.2486e-04, 3.

: 

In [346]:
## Il faut utiliser des pytorch container comme dans ConvNet pour définir les layers ??

In [ ]:
## solution to merge models into a single one !!

class MyModel(nn.Module):
    def __init__(self, config):
        super(MyModel, self).__init__()

        data_module = DataModule_Learning(config)
        model = ContrastiveLearnerFusion(config,
                                 sample_data=data_module)
        
        modules_encoder = []
        for name, module in model.named_modules():
            modules_encoder.append(
                (name, module)
            )
        #for name, param in model.backbones[0].encoder.named_parameters():
        #    self.name = 
        #self.fc1 = model.backbones[0].encoder
        #self.fcs = model.projection_head
        self.net = nn.Sequential(OrderedDict(modules_encoder))
        
    def forward(self, x):
        x = self.fc1(x)
        for l in self.fcs:
            x = F.relu(l(x))
        return x
            
net = MyModel(config)